# Import Libraries

In [1]:
import pandas as pd
# change to an interactive plotting backend
pd.options.plotting.backend = "plotly"
from IPython import get_ipython

# custom functions
import custom_functions as funcs

# Retrieve the Data with Daily Granularity
First, we run a custom script to check if the `PostgreSQL` server is started or to start it.

In [2]:
get_ipython().system('bash $MYBINS/pg_server_status')


==> Postgres server is already running.

PostgreSQL binary:/usr/local/Cellar/postgresql/13.3/bin/postgres
Server directory:/usr/local/var/postgres


As explained in the previous chapter, inserting the data into an SQL database enables faster operations on the data - which in this case amount in mostinstances to `GROUP BY`. Data is hence served much faster.

Now, two datasets are retrieved:

* The **daily outflow for the whole municipality** of Milan.
* The **daily outflow for each station** in the city.
 
The custom function`retrieve_bike_flow` has an argument, `time_column` which has a predefined value: `{"data_prelievo": "giorno_partenza"}`. This implies that the function will retrieve data aggregated on the time by which the bikes leave the station and not when they are returned once the ride is finished. In other words,by default the daily outflow will be retrieved.

Besides, the parameter `trunc` is set to default to `day`: this will mean that the `time_column` will be aggregated to the **daily level** - but this parameter can be set to other time units, such as `hour` to retrive hourly data. Regardless, we expect the daily aggregation to yield better overall results, as the hourly data becomes quite more noisy, especially when considering a single time series for each station.

In [3]:
# when no argument other than the table name is specified, daily data for Milan is retrieved
daily_outflow = funcs.retrieve_bike_flow(table="bikemi_2019")
daily_outflow.head()

,count
giorno_partenza,
2019-01-01,831
2019-01-02,3512
2019-01-03,4126
2019-01-04,4201
2019-01-05,2875


In [6]:
# let's get some info
daily_outflow.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2019-01-01 to 2020-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   count   366 non-null    int64
dtypes: int64(1)
memory usage: 5.7 KB


The data already has a `DateTimeIndex`, which can easily be indexed. It has no null counts, but has 366 observations (instead of 365), but we will be back to this in a minute.

In [11]:
# display NaNs
daily_outflow.isna().sum()

count    0
dtype: int64

The data does not have `NA` values, either.

In [4]:
daily_outflow.plot(
    title="Bikes Daily Outflow (2019)"
)

Exploiting the `datetime` index, we can extract some new columns containing the information about the day and the month.

In [14]:
daily_outflow.index.day

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
            ...
            23, 24, 25, 26, 27, 28, 29, 30, 31,  1],
           dtype='int64', name='giorno_partenza', length=366)

In [5]:
# when the station_columns is specified, data for each station is retrieved
station_daily_outflow = (
    funcs.retrieve_bike_flow(
        table="bikemi_2019",
        station_column={"nome_stazione_prelievo": "stazione_partenza"}
    )
    .pipe(funcs.pivot_bike_flow, cols="stazione_partenza")
    .convert_dtypes()  # make double into integers
)

station_daily_outflow.head()

,Accursio - Portello,Adamello - Lorenzini,Alserio - Cola Montano,Amendola M1,Angilberto - Comacchio,Aquileia,Arcimboldi - Innovazione,Arcivescovado,Arco della Pace 1 - Bertani,Arco della Pace 2 - Pagano,...,Verrocchio - Juvara,Vesuvio - Lipari,Vigliani - Scarampo,Villapizzone FFSS - Lambruschini,Vittor Pisani,XXII Marzo,XXIV Maggio,XXV Aprile,Zara M3,Zavattari - Tempesta
giorno_partenza,,,,,,,,,,,,,,,,,,,,,
2019-01-01,2,1,7,2,2,1,3,10,4,6,...,2,<NA>,<NA>,1,6,1,3,14,<NA>,2
2019-01-02,10,7,13,8,2,4,7,29,22,14,...,12,4,2,2,19,7,11,39,3,2
2019-01-03,13,5,9,13,2,8,6,26,15,21,...,15,14,4,3,32,6,14,30,7,5
2019-01-04,10,8,15,13,2,6,1,19,21,23,...,9,13,1,1,29,11,15,45,8,3
2019-01-05,14,4,5,8,1,14,2,22,19,20,...,10,11,1,3,14,8,12,29,6,2


## Filter out invalid data
Before proceeding with the time series analysis, we first need to check out if theretrieved data belongs to the 2019. (Quick note: since the index is a datetime index,we can subset the data very easily by using, for example, this notation:`daily_outflow.loc["2019-10"]`.)

The quickest way of doing it is by checking the length of our datasets:


In [6]:
print(
    f"Lenght of `daily_outflow` is {len(daily_outflow)}\n",
    f"Length of `station_daily_outflow` is {len(station_daily_outflow)}"
)

Lenght of `daily_outflow` is 366 Length of `station_daily_outflow` is 366


 We can further inspect the issue by looking at the last values of either dataset:

In [7]:
daily_outflow.tail()

,count
giorno_partenza,
2019-12-28,1944
2019-12-29,1269
2019-12-30,2698
2019-12-31,2126
2020-01-01,31


 How come 31 trips from the year 2020 sneaked in? The data comes from several `.xlsx` files that were put into a local database, to make retrieval faster. In other words, these observations were already there. If the analysis is performed by concatenating all tables for all years, the problem will be fixed. In the meantime, dropping these observations will do.

In [8]:
daily_outflow = daily_outflow.loc[daily_outflow.index != "2020-01"]
station_daily_outflow = station_daily_outflow.loc[station_daily_outflow.index != "2020-01"]

# Export data as `.csv`

Finally, let's export the data as lightweight `.csv` files.


In [9]:
daily_outflow.to_csv("/data/bikemi_csv/daily_outflow.csv")
station_daily_outflow.to_csv("/data/bikemi_csv/station_daily_outflow.csv")

# Hourly Data

In [4]:
# we can also extract the hourly data at the aggregate level:
hourly_outflow = funcs.retrieve_bike_flow(table="bikemi_2019", trunc="hour")

In [10]:
hourly_outflow.plot(
    title = "Hourly Bike Count in Milan, 2019"
)

In [9]:
hourly_outflow.loc[]

KeyError: 'False: boolean label can not be used without a boolean index'